# DuckDB Python Client Basics

In [1]:
import duckdb
import pandas as pd
import numpy as np
import polars as pl
import pyarrow as pa
import os.path

duckdb.__version__

'0.10.0'

# Basic Python API Usage

In [2]:
duckdb.sql("SELECT 42").show()

┌───────┐
│  42   │
│ int32 │
├───────┤
│    42 │
└───────┘



In [3]:
relation = duckdb.sql(
    "SELECT 42 AS i"
)

(duckdb
 .sql("SELECT i * 2 AS k FROM relation")
 .show()
)

┌───────┐
│   k   │
│ int32 │
├───────┤
│    84 │
└───────┘



# Data Input

In [4]:
ls

Historic_Districts_20240426.csv  duckdb-python-basics.ipynb
data/                            duckdb-python-sql.ipynb


In [5]:
# duckdb.read_csv()
path = "Historic_Districts_20240426.csv"
duckdb.read_csv(path, date_format= "%m/%d/%Y")

┌──────────────────────┬─────────┬───────────┬──────────┬───┬────────────┬────────────┬───────────────┬───────────────┐
│       the_geom       │ BOROUGH │ LP_NUMBER │ CURRENT_ │ … │  DESDATE   │  CALDATE   │  Shape_Leng   │  Shape_Area   │
│       varchar        │ varchar │  varchar  │ varchar  │   │    date    │    date    │    double     │    double     │
├──────────────────────┼─────────┼───────────┼──────────┼───┼────────────┼────────────┼───────────────┼───────────────┤
│ MULTIPOLYGON (((-7…  │ QN      │ LP-02040  │ Yes      │ … │ 1999-06-29 │ 1999-04-13 │ 11203.1997224 │ 4067523.25047 │
│ MULTIPOLYGON (((-7…  │ MN      │ LP-01985  │ Yes      │ … │ 1998-05-05 │ NULL       │ 410.860306645 │ 10652.4239136 │
│ MULTIPOLYGON (((-7…  │ MN      │ LP-01901  │ Yes      │ … │ 1993-02-25 │ NULL       │ 4995.47146205 │ 1069203.49477 │
│ MULTIPOLYGON (((-7…  │ BK      │ LP-00989  │ Yes      │ … │ 1978-07-11 │ NULL       │ 1240.77043794 │  78546.856215 │
│ MULTIPOLYGON (((-7…  │ BK      │ LP-02

My preferred method - `duckdb.sql()`

In [6]:
# duckdb.sql(SELECT)
duckdb.sql(
    """
    SELECT *
    FROM Historic_Districts_20240426.csv
    LIMIT 10
    """
)

┌──────────────────────┬─────────┬───────────┬──────────┬───┬────────────┬────────────┬───────────────┬───────────────┐
│       the_geom       │ BOROUGH │ LP_NUMBER │ CURRENT_ │ … │  DESDATE   │  CALDATE   │  Shape_Leng   │  Shape_Area   │
│       varchar        │ varchar │  varchar  │ varchar  │   │    date    │    date    │    double     │    double     │
├──────────────────────┼─────────┼───────────┼──────────┼───┼────────────┼────────────┼───────────────┼───────────────┤
│ MULTIPOLYGON (((-7…  │ QN      │ LP-02040  │ Yes      │ … │ 1999-06-29 │ 1999-04-13 │ 11203.1997224 │ 4067523.25047 │
│ MULTIPOLYGON (((-7…  │ MN      │ LP-01985  │ Yes      │ … │ 1998-05-05 │ NULL       │ 410.860306645 │ 10652.4239136 │
│ MULTIPOLYGON (((-7…  │ MN      │ LP-01901  │ Yes      │ … │ 1993-02-25 │ NULL       │ 4995.47146205 │ 1069203.49477 │
│ MULTIPOLYGON (((-7…  │ BK      │ LP-00989  │ Yes      │ … │ 1978-07-11 │ NULL       │ 1240.77043794 │  78546.856215 │
│ MULTIPOLYGON (((-7…  │ BK      │ LP-02

# DataFrames

In [7]:
ls

Historic_Districts_20240426.csv  duckdb-python-basics.ipynb
data/                            duckdb-python-sql.ipynb


In [8]:
# pandas
df = pd.DataFrame({"a": [42]})

duckdb.sql("SELECT * FROM df")

┌───────┐
│   a   │
│ int64 │
├───────┤
│    42 │
└───────┘

In [9]:
# polars
df = pl.DataFrame({"a": [42]})

duckdb.sql("SELECT * FROM df")

┌───────┐
│   a   │
│ int64 │
├───────┤
│    42 │
└───────┘

In [10]:
# pyarrow
df = pa.Table.from_pydict({"a": [42]})

duckdb.sql("SELECT * FROM df")

┌───────┐
│   a   │
│ int64 │
├───────┤
│    42 │
└───────┘

# Result Conversion

In [11]:
# Python objects
print(type(duckdb.sql("SELECT 42").fetchall()))

# NumPy Arrays
print(type(duckdb.sql("SELECT 42").fetchnumpy())) 

# Pandas DataFrame
print(type(duckdb.sql("SELECT 42").df()))

# Polars DataFrame
print(type(duckdb.sql("SELECT 42").pl()))

# Arrow Table
print(type(duckdb.sql("SELECT 42").arrow()))

<class 'list'>
<class 'dict'>
<class 'pandas.core.frame.DataFrame'>
<class 'polars.dataframe.frame.DataFrame'>
<class 'pyarrow.lib.Table'>


In [12]:
# Example: pandas DataFrame
duckdb.sql("SELECT 42 AS x").df()

,x
0,42


# Writing Data to Disk

In [13]:
# Write to a Parquet file
duckdb.sql("SELECT 42").write_parquet("data/out.parquet") 

# Write to a CSV file
duckdb.sql("SELECT 42").write_csv("data/out.csv")   

# Copy to a Parquet file
duckdb.sql("COPY (SELECT 42) TO 'data/out.parquet'")      

In [21]:
ls data/

out.csv      out.parquet


# Connection Options

## Using an In-Memory Database

In [15]:
con = duckdb.connect()

(con
 .sql("SELECT 42 AS x")
 .show()
)

┌───────┐
│   x   │
│ int32 │
├───────┤
│    42 │
└───────┘



A connection to a persistent database can be created using the `connect` function.

## Persistent Storage

In [16]:
con = duckdb.connect("file.db")

con.sql("CREATE TABLE IF NOT EXISTS integers (i INTEGER)")
con.sql("INSERT INTO integers VALUES (42)")

con.sql("SELECT * FROM integers").show()

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
└───────┘



In [17]:
con = duckdb.connect("file.db")

con.sql("SELECT * FROM integers").show()

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
└───────┘



In [18]:
# create a connection to a file called 'file.db'
con = duckdb.connect("file.db")

con.sql("INSERT INTO integers VALUES (24)")

# query the table
con.table("integers").show()

# explicitly close the connection
con.close()
# Note: connections also closed implicitly when they go out of scope

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
│    24 │
└───────┘



In [19]:
ls

Historic_Districts_20240426.csv  duckdb-python-sql.ipynb
data/                            file.db
duckdb-python-basics.ipynb


In [20]:
with duckdb.connect("file.db") as con:
    con.sql("INSERT INTO integers VALUES (42)")
    con.table("integers").show()
    # the context manager closes the connection automatically

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
│    24 │
│    42 │
└───────┘

